# Cat Vs Dog - Fine Tuning
#### VGG16 Pre Trained Model (Transfer Learning)

**NOTE: This technique is so expensive that you should only attempt it if you
have access to a GPU—it’s absolutely intractable on CPU. If you can’t run your
code on GPU, then the without augmented is the way to go.**

Download Datasets: https://www.kaggle.com/c/dogs-vs-cats/data


***NOTE: IF YOU JUST DOWNLOADED DATA, FIRST RUN "Copying images for CNN Examples.ipynb"  FOR IMAGE MANIPULATION:***

In [ ]:
import os, numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

from tensorflow.keras.applications import VGG16
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers

In [ ]:
base_dir = '/home/selim/Documents/Datasets/CatVsDog/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

### Instantiating the VGG16 convolutional base

In [ ]:
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

In [ ]:
conv_base.summary()

### Freezing all layers up to a specific one

In [ ]:
conv_base.trainable = True
set_trainable = False

for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

### Adding a densely connected classifier on top of the convolutional base

In [ ]:
model = models.Sequential()

model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

### Training the model end to end with a frozen convolutional base

In [ ]:
# Note that the validation data shouldn’t be augmented!
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


test_datagen = ImageDataGenerator(rescale=1./255)

# Resizes all images to 150 × 150
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

### Fine-tuning the model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-5), metrics=['acc'])

history = model.fit(train_generator, 
                              steps_per_epoch=100, epochs=100, 
                              validation_data=validation_generator, 
                              validation_steps=50)


In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

### Saving Model

In [ ]:
model.save('catVsDogFineTuning.h5')

###  Plotting the results

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend();